# 6. Speech-to-Text and Text-to-Speech

In [10]:
import tomli, os
with open("../.streamlit/secrets.toml","rb") as f:
    secrets = tomli.load(f)
os.environ["OPENAI_API_KEY"] = secrets["OPENAI_API_KEY"]

## 6.1. Transcription

Whisper can be downloaded locally to perform transcription without having to send your audio files over the internet. This is useful for privacy reasons, or when you have a slow or no internet connection.
```
pip install -U openai-whisper
```

In [4]:
import os
os.listdir('../data/audio/')
# os.getcwd()

['enYann-tale_of_two_cities.mp3']

In [5]:
# Listen to the audio before transcribing
from IPython.display import Audio
file = '../data/audio/enYann-tale_of_two_cities.mp3'
Audio(file)

In [11]:
import whisper
model = whisper.load_model("base")
result = model.transcribe(file) # fp16=False to use FP32 instead
print(result["text"])

c:\Users\ydebray\Downloads\gpt-programming-book\env\lib\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, it was the season of light, it was the season of darkness, it was the spring of hope, it was the winter of despair, we had everything before us, we had nothing before us, we were all going direct to heaven, we were all going direct the other way, in short the period was so far like the present period that some of its noisiest authorities insisted on its being received for good or for evil in the superlative degree of comparison only.


You can also use Whisper through the transcription API  of openAI. Currently, there is no difference between the open source version of Whisper and the version available through the API. However, through the API, OpenAI offers an optimized inference process which makes running Whisper through the API much faster than doing it through other means. 

In [7]:
import openai
from pathlib import Path
file_path = Path(file)
transcription = openai.audio.transcriptions.create(model="whisper-1", file=file_path)
transcription.text

'It was the best of times. It was the worst of times. It was the age of wisdom. It was the age of foolishness It was the the epoch of belief It was the epoch of incredulity. It was the season of light. It was the season of darkness It was the spring of hope. It was the winter of despair. We had everything before us We had nothing before us. We were all going direct to heaven We were all going direct the other way In short the period was so far like the present period that some of its noisiest authorities Insisted on its being received for good or for evil in the superlative degree of comparison only'

## 6.2. Voice synthesis

In [9]:
import openai

speech_file_path = "../data/audio/speech.mp3"
response = openai.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input="The quick brown fox jumped over the lazy dog."
)
response.stream_to_file(speech_file_path)


C:\Users\ydebray\AppData\Local\Temp\ipykernel_12216\935334227.py:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [10]:
Audio(speech_file_path)

## 6.3. Application: Daily tech podcast

In [2]:
import requests, datetime
url = "https://techcrunch.com/feed"
date = datetime.datetime.now().strftime("%Y-%m-%d") # date in format YYYY-MM-DD
response = requests.get(url)
with open(f"../data/rss/techcrunch_{date}.xml", "wb") as f:
    f.write(response.content)

In [4]:
import xml.etree.ElementTree as ET
# Parse the XML document
tree = ET.fromstring(response.content)
# Get the channel element
channel = tree.find('channel')
# Print the channel title
print(f"Channel Title: {channel.find('title').text}")
# Print the channel description
print(f"Channel Description: {channel.find('description').text}")
# Print the channel link
print(f"Channel Link: {channel.find('link').text}")
# Print the channel last build date
print(f"Last Build Date: {channel.find('lastBuildDate').text}")

Channel Title: TechCrunch
Channel Description: Startup and Technology News
Channel Link: https://techcrunch.com/
Last Build Date: Sat, 27 Apr 2024 20:20:46 +0000


In [5]:
len(channel.findall('item'))

20

In [6]:
# Print the items
print("\nItems:\n")
for item in channel.findall('item')[:3]:
    title = item.find('title').text
    link = item.find('link').text
    description = item.find('description').text.strip().replace('<p>© 2024 TechCrunch. All rights reserved. For personal use only.</p>','')
    print(f"Title: {title}")
    print(f"Link: {link}")
    print(f"Description: {description}")
    print("-" * 80)


Items:

Title: TikTok faces a ban in the US, Tesla profits drop and healthcare data leaks
Link: https://techcrunch.com/2024/04/27/tiktok-faces-a-ban-in-the-us-tesla-profits-drop-and-healthcare-data-leaks/
Description: <p>Welcome, folks, to Week in Review (WiR), TechCrunch&#8217;s regular newsletter covering this week&#8217;s noteworthy happenings in tech. TikTok&#8217;s fate in the U.S. looks uncertain after President Joe Biden signed a bill that included a deadline for ByteDance, TikTok&#8217;s parent company, to divest itself of TikTok within nine months or face a ban on distributing it [&#8230;]</p>

--------------------------------------------------------------------------------
Title: Will a TikTok ban impact creator economy startups? Not really, founders say
Link: https://techcrunch.com/2024/04/27/will-a-tiktok-ban-impact-creator-economy-startups-not-really-founders-say/
Description: <p>President Joe Biden signed a bill on Wednesday that could ban TikTok – for real this time. Af

In [8]:
from bs4 import BeautifulSoup
import requests
item = channel.find('item')
title = item.find('title').text
link = item.find('link').text
html = requests.get(link).text
soup = BeautifulSoup(html, "html.parser")
# extract only the text from the class article-content
text = soup.find(class_="article-content").get_text()
with open(f"../data/txt/{title}.txt", "w", encoding="utf-8") as f:
    f.write(text)

In [9]:
print(text[0:42])


Welcome, folks, to Week in Review (WiR), 


In [12]:
import openai
speech_file_path = f"../data/audio/{title}.mp3"
response = openai.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input=text
)
response.stream_to_file(speech_file_path)

C:\Users\ydebray\AppData\Local\Temp\ipykernel_18016\3019291008.py:8: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [46]:
def summarize(text):
    inst = '''Resume l'article suivant en moins de 1000 caracteres:''' 
    completion = openai.chat.completions.create(
        model='gpt-3.5-turbo-1106',
        messages= [
            {'role': 'system', 'content': inst },
            {'role': 'user', 'content': text }]
    )
    return completion.choices[0].message.content

In [47]:
summarize(text)

"Cette semaine, TikTok s'est retrouvé dans l'incertitude aux États-Unis après que le Président Joe Biden a signé une loi fixant un délai de neuf mois pour la maison mère, ByteDance, pour se séparer de l'application. Par ailleurs, la filiale de UnitedHealth, Change Healthcare, a confirmé une importante fuite de données suite à une cyberattaque. Tesla a enregistré une chute de 55% de ses profits, tandis qu'Amazon a lancé un nouvel abonnement de livraison illimitée pour les courses. Stripe a également annoncé une expansion de ses services. D'autres actualités et analyses sont également couvertes dans cette édition de WiR."